In [57]:
import sys
import numpy as np
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

In [67]:
model_name = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_name)

In [63]:
# Question 1

input_text = "I just discovered the course. Can I join now?"
embedding = embedding_model.embed(input_text)
embedding_out = list(embedding)[0] # Selecting the first embedding from the generator

print(len(embedding_out), min(embedding_out), max(embedding_out))

512 -0.11726373551188797 0.13307955253468784


In [64]:
# Question 2

print("Norm of embedding_out: ", np.linalg.norm(embedding_out))
print("Dot product of embedding_out with itself: ", embedding_out.dot(embedding_out))

doc = "Can I still join the course after the start date?"
query = "I just discovered the course. Can I join now?"

query_embedding = list(embedding_model.embed(query))[0]
doc_embedding = list(embedding_model.embed(doc))[0]

print("Dot Product of query and doc: ", query_embedding.dot(doc_embedding))

Norm of embedding_out:  1.0
Dot product of embedding_out with itself:  1.0
Dot Product of query and doc:  0.9008528856818037


In [65]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]


In [80]:
# Question 3

query = "I just discovered the course. Can I join now?"
query_embedding = list(embedding_model.embed(query))[0]

# Create a list to collect embeddings, then convert to numpy matrix
doc_embeddings_list = []
for doc in documents:
    doc_embedding = list(embedding_model.embed(doc["text"]))[0]
    doc_embeddings_list.append(doc_embedding)

# Convert list to numpy matrix V (shape: num_docs x embedding_dim)
V = np.array(doc_embeddings_list)
print(f"Matrix V shape: {V.shape}")

# Compute cosine similarities using matrix multiplication
# Since embeddings are normalized (norm=1), dot product = cosine similarity
cosine_similarities = V.dot(query_embedding)

print(cosine_similarities)
# Find the document with highest similarity
max_idx = np.argmax(cosine_similarities)
print(f"Most similar document index={max_idx}: {cosine_similarities[max_idx]}")

Matrix V shape: (5, 512)
[0.76296845 0.81823783 0.80853973 0.71330786 0.73044994]
Most similar document index=1: 0.8182378325403555


In [81]:
# Question 4

query = "I just discovered the course. Can I join now?"
query_embedding = list(embedding_model.embed(query))[0]

# Create a list to collect embeddings, then convert to numpy matrix
doc_embeddings_list = []
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    doc_embedding = list(embedding_model.embed(full_text))[0]
    doc_embeddings_list.append(doc_embedding)

# Convert list to numpy matrix V (shape: num_docs x embedding_dim)
V = np.array(doc_embeddings_list)
print(f"Matrix V shape: {V.shape}")

# Compute cosine similarities using matrix multiplication
# Since embeddings are normalized (norm=1), dot product = cosine similarity
cosine_similarities = V.dot(query_embedding)

print(cosine_similarities)
# Find the document with highest similarity
max_idx = np.argmax(cosine_similarities)
print(f"Most similar document index={max_idx}: {cosine_similarities[max_idx]}")

Matrix V shape: (5, 512)
[0.85145431 0.84365941 0.84082872 0.77551577 0.8086008 ]
Most similar document index=0: 0.8514543121225202


In [58]:
# Question 5
min_dim = sys.maxsize
model_info = None
for i in TextEmbedding.list_supported_models():
    if min_dim > i["dim"]:
        min_dim = i["dim"]
        model_info = i

print(min_dim, model_info)

384 {'model': 'BAAI/bge-small-en', 'sources': {'hf': 'Qdrant/bge-small-en', 'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz', '_deprecated_tar_struct': True}, 'model_file': 'model_optimized.onnx', 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.', 'license': 'mit', 'size_in_GB': 0.13, 'additional_files': [], 'dim': 384, 'tasks': {}}


In [84]:
# Question 6

import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [91]:
model_handle_name = "BAAI/bge-small-en"
qd_client = QdrantClient(host="localhost", port=6333)
EMBEDDING_DIMENSIONALITY = 384

# Define the collection name
collection_name = "zoomcamp-rag"

# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

points = []
for id, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=text, model=model_handle_name), #embed text locally 
        payload=doc #save all needed metadata fields
    )
    points.append(point)


qd_client.upsert(
    collection_name=collection_name,
    points=points
)



UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [99]:
def search(query, limit=1):
    return qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally
            text=query,
            model=model_handle_name 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

out = search("I just discovered the course. Can I join now?")
print(out.points[0].score, out.points[0].payload)


0.8703172 {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}


Testing commands

In [93]:
# Testing with direct cosine similarity

model_handle_name = "BAAI/bge-small-en"
embedding_model = TextEmbedding(model_handle_name)

query = "I just discovered the course. Can I join now?"
query_embedding = list(embedding_model.embed(query))[0]

doc_embeddings_list = []
for i in documents:
    text = doc['question'] + ' ' + doc['text']
    doc_embedding = list(embedding_model.embed(text))[0]
    doc_embeddings_list.append(doc_embedding)

V = np.array(doc_embeddings_list)

cosine_similarities = V.dot(query_embedding)

# Find the document with highest similarity
max_idx = np.argmax(cosine_similarities)
print(f"Most similar document index={max_idx}: {cosine_similarities[max_idx]}")
print(documents[max_idx])

Most similar document index=0: 0.8067650198936462
{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', 'section': 'General course-related questions', 'question': 'How do I sign up?', 'course': 'machine-learning-zoomcamp'}
